In [4]:
# Import required libraries for data handling and mapping
import os
from glob import glob

import folium
import geopandas as gpd
import pandas as pd


In [5]:
# Import, append, and deduplicate all crime incident files (2019-2025)
data_folder = '../data/raw/'
csv_files = sorted(glob(os.path.join(data_folder, 'Crime_Incidents_in_20*.csv')))
print(f'Found CSV files: {csv_files}')
if not csv_files:
    raise FileNotFoundError('No crime incident CSV files found in the expected folder.')
dfs = [pd.read_csv(f) for f in csv_files]
crime_incidents = pd.concat(dfs, ignore_index=True).drop_duplicates()

Found CSV files: ['../data/raw/Crime_Incidents_in_2019.csv', '../data/raw/Crime_Incidents_in_2020.csv', '../data/raw/Crime_Incidents_in_2021.csv', '../data/raw/Crime_Incidents_in_2022.csv', '../data/raw/Crime_Incidents_in_2023.csv', '../data/raw/Crime_Incidents_in_2024.csv', '../data/raw/Crime_Incidents_in_2025.csv']


In [6]:
# Standardize column names in crime_incidents DataFrame (lowercase, underscores)
crime_incidents.columns = [col.strip().lower().replace(' ', '_') for col in crime_incidents.columns]

# rename report_dat as report_date
crime_incidents = crime_incidents.rename(columns={'report_dat': 'report_date'})

# reformat report_date to datetime
crime_incidents['report_date'] = pd.to_datetime(crime_incidents['report_date'], errors='coerce')
# also start date and end date
crime_incidents['start_date'] = pd.to_datetime(crime_incidents['start_date'], errors='coerce')
crime_incidents['end_date'] = pd.to_datetime(crime_incidents['end_date'], errors='coerce')

# add a new column called year drawn from report_date without any numbers after decimal
crime_incidents['year'] = crime_incidents['report_date'].dt.year.astype(str)

# Show standardized column names
crime_incidents.columns.tolist()

['x',
 'y',
 'ccn',
 'report_date',
 'shift',
 'method',
 'offense',
 'block',
 'xblock',
 'yblock',
 'ward',
 'anc',
 'district',
 'psa',
 'neighborhood_cluster',
 'block_group',
 'census_tract',
 'voting_precinct',
 'latitude',
 'longitude',
 'bid',
 'start_date',
 'end_date',
 'objectid',
 'octo_record_id',
 'year']

In [7]:
# View the first few rows of the combined and deduplicated crime incidents dataset
crime_incidents.head()

,x,y,ccn,report_date,shift,method,offense,block,xblock,yblock,...,census_tract,voting_precinct,latitude,longitude,bid,start_date,end_date,objectid,octo_record_id,year
0,400134.00,130213.00,19133977,2019-07-31 02:06:18+00:00,EVENING,GUN,ASSAULT W/DANGEROUS WEAPON,3301 - 3699 BLOCK OF 6TH STREET SE,400134.00,130213.00,...,9804.0,Precinct 122,38.839713,-76.998457,NaN,2019-07-30 22:42:17+00:00,2019-07-30 22:55:26+00:00,717509953,NaN,2019
1,397228.00,137798.00,19046953,2019-03-20 16:35:01+00:00,DAY,OTHERS,THEFT/OTHER,1300 - 1399 BLOCK OF 14TH STREET NW,397228.00,137798.00,...,5203.0,Precinct 17,38.908037,-77.031960,NaN,2019-03-20 15:30:24+00:00,2019-03-20 15:45:03+00:00,717509954,NaN,2019
2,397705.17,141966.76,19051853,2019-03-28 02:08:24+00:00,EVENING,OTHERS,THEFT/OTHER,4500 - 4599 BLOCK OF GEORGIA AVENUE NW,397705.17,141966.76,...,2400.0,Precinct 46,38.945592,-77.026472,NaN,2019-03-27 18:57:00+00:00,2019-03-28 02:17:53+00:00,717509955,NaN,2019
3,402152.00,140253.00,19206304,2019-11-18 10:39:13+00:00,MIDNIGHT,OTHERS,THEFT F/AUTO,2000 - 2199 BLOCK OF JACKSON STREET NE,402152.00,140253.00,...,9400.0,Precinct 70,38.930155,-76.975181,NaN,2019-11-17 19:00:24+00:00,2019-11-18 07:45:26+00:00,717509956,NaN,2019
4,396307.07,137186.46,19079495,2019-05-09 12:21:18+00:00,DAY,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF K STREET NW,396307.07,137186.46,...,10700.0,Precinct 17,38.902525,-77.042574,GOLDEN TRIANGLE,2019-05-08 22:30:40+00:00,2019-05-09 10:45:37+00:00,717509960,NaN,2019


In [8]:
# output as a csv
crime_incidents.to_csv(os.path.join(data_folder, 'crime_incidents.csv'), index=False)

In [9]:
# list every value variant in the offense field
offense_variants = crime_incidents['offense'].unique()
offense_variants

array(['ASSAULT W/DANGEROUS WEAPON', 'THEFT/OTHER', 'THEFT F/AUTO',
       'ROBBERY', 'MOTOR VEHICLE THEFT', 'SEX ABUSE', 'HOMICIDE',
       'BURGLARY', 'ARSON'], dtype=object)

In [10]:
# Count incidents grouped by offense and year, with years as columns
if 'year' not in crime_incidents.columns or 'offense' not in crime_incidents.columns:
    raise ValueError("Missing 'year' or 'offense' column in crime_incidents DataFrame.")
crime_offense_yearly = crime_incidents.groupby(['offense', 'year']).size().unstack(fill_value=0)
crime_offense_yearly

year,2019,2020,2021,2022,2023,2024,2025
offense,,,,,,,
ARSON,8,13,4,4,11,4,4
ASSAULT W/DANGEROUS WEAPON,1573,1632,1658,1379,1404,1025,553
BURGLARY,1274,1443,1174,1049,1092,1005,459
HOMICIDE,166,198,222,203,274,187,101
MOTOR VEHICLE THEFT,2187,3260,3479,3742,6791,5128,2979
ROBBERY,2235,1998,2038,2074,3463,2110,917
SEX ABUSE,199,178,189,182,192,142,57
THEFT F/AUTO,10741,8285,8655,7762,7750,6679,3840
THEFT/OTHER,15572,10929,10892,10765,13267,13001,7330


In [11]:
# Categorize offenses into violent, property, minor property theft, and other
def categorize_offense(offense):
    violent = [
        'HOMICIDE',
        'ASSAULT W/DANGEROUS WEAPON', 
        'SEX ABUSE',
        'ROBBERY'
    ]
    property_crimes = [
        'MOTOR VEHICLE THEFT',
        'ARSON',
        'BURGLARY'
    ]
    minor_property_theft = [
        'THEFT F/AUTO',
        'THEFT/OTHER'
    ]
    
    if offense in violent:
        return 'violent'
    elif offense in property_crimes:
        return 'property'
    elif offense in minor_property_theft:
        return 'minor property theft'
    else:
        return 'other'

crime_incidents['crime_type'] = crime_incidents['offense'].apply(categorize_offense)

In [12]:
# Create a comparable 'year-to-date' pivot table for each year (2019-2025) using the min/max date in 2025 (month/day only)
max_2025_date = crime_incidents.loc[crime_incidents['year'] == '2025', 'report_date'].max()
min_2025_date = crime_incidents.loc[crime_incidents['year'] == '2025', 'report_date'].min()
if pd.isnull(max_2025_date) or pd.isnull(min_2025_date):
    raise ValueError("No valid dates found for 2025 in the dataset.")
# Get month/day for min and max dates
min_md = (min_2025_date.month, min_2025_date.day)
max_md = (max_2025_date.month, max_2025_date.day)
# Filter for incidents in each year between min_md and max_md (month/day only)
def is_within_period(row):
    if pd.isnull(row['report_date']):
        return False
    md = (row['report_date'].month, row['report_date'].day)
    return (md >= min_md) and (md <= max_md)
filtered = crime_incidents[crime_incidents.apply(is_within_period, axis=1)]
# Only keep years 2019-2025
filtered = filtered[filtered['year'].isin([str(y) for y in range(2019, 2026)])]
# Pivot: offenses as rows, years as columns
crime_offense_ytd = filtered.groupby(['offense', 'year']).size().unstack(fill_value=0)

# Print the date range used for each year
for year in range(2019, 2026):
    print(f"Year {year}: {year}-{min_md[0]:02d}-{min_md[1]:02d} to {year}-{max_md[0]:02d}-{max_md[1]:02d}")

crime_offense_ytd.head()

Year 2019: 2019-01-01 to 2019-08-19
Year 2020: 2020-01-01 to 2020-08-19
Year 2021: 2021-01-01 to 2021-08-19
Year 2022: 2022-01-01 to 2022-08-19
Year 2023: 2023-01-01 to 2023-08-19
Year 2024: 2024-01-01 to 2024-08-19
Year 2025: 2025-01-01 to 2025-08-19


year,2019,2020,2021,2022,2023,2024,2025
offense,,,,,,,
ARSON,7,11,4,3,8,3,4
ASSAULT W/DANGEROUS WEAPON,1005,1038,990,892,922,682,553
BURGLARY,815,1020,662,680,696,580,459
HOMICIDE,106,118,127,131,170,114,101
MOTOR VEHICLE THEFT,1316,1793,2097,2157,4620,3047,2979


In [13]:
# Repeat the above but use August 11 as the low date
max_2025_date = crime_incidents.loc[crime_incidents['year'] == '2025', 'report_date'].max()
min_2025_date = pd.Timestamp(year=2025, month=8, day=11)
if pd.isnull(max_2025_date) or pd.isnull(min_2025_date):
    raise ValueError("No valid dates found for 2025 in the dataset.")
# Get month/day for min and max dates
min_md = (min_2025_date.month, min_2025_date.day)
max_md = (max_2025_date.month, max_2025_date.day)
# Filter for incidents in each year between min_md and max_md (month/day only)
def is_within_period(row):
    if pd.isnull(row['report_date']):
        return False
    md = (row['report_date'].month, row['report_date'].day)
    return (md >= min_md) and (md <= max_md)
filtered = crime_incidents[crime_incidents.apply(is_within_period, axis=1)]
# Only keep years 2019-2025
filtered = filtered[filtered['year'].isin([str(y) for y in range(2019, 2026)])]
# Pivot: offenses as rows, years as columns
crime_offense_since_aug11 = filtered.groupby(['offense', 'year']).size().unstack(fill_value=0)

# Print the date range used for each year
for year in range(2019, 2026):
    print(f"Year {year}: {year}-{min_md[0]:02d}-{min_md[1]:02d} to {year}-{max_md[0]:02d}-{max_md[1]:02d}")

crime_offense_since_aug11.head()

Year 2019: 2019-08-11 to 2019-08-19
Year 2020: 2020-08-11 to 2020-08-19
Year 2021: 2021-08-11 to 2021-08-19
Year 2022: 2022-08-11 to 2022-08-19
Year 2023: 2023-08-11 to 2023-08-19
Year 2024: 2024-08-11 to 2024-08-19
Year 2025: 2025-08-11 to 2025-08-19


year,2019,2020,2021,2022,2023,2024,2025
offense,,,,,,,
ARSON,0,0,0,1,0,0,0
ASSAULT W/DANGEROUS WEAPON,38,29,39,34,33,16,17
BURGLARY,36,33,36,24,29,36,21
HOMICIDE,8,3,9,2,7,2,2
MOTOR VEHICLE THEFT,64,86,75,72,179,145,77


In [11]:
# Create a comprehensive DataFrame showing crime counts by offense type, ward, and year
crime_by_ward_year_offense = crime_incidents.groupby(['ward', 'year', 'offense']).size().reset_index(name='count')

# Create separate DataFrames for each offense type by ward and year
offense_types = crime_incidents['offense'].unique()
offense_dataframes = {}

print(f"Creating DataFrames for {len(offense_types)} offense types...")

for offense in offense_types:
    # Filter data for this specific offense
    offense_data = crime_incidents[crime_incidents['offense'] == offense]
    
    # Create pivot table: wards as rows, years as columns
    offense_pivot = offense_data.groupby(['ward', 'year']).size().unstack(fill_value=0)
    
    # Store in dictionary
    offense_dataframes[offense] = offense_pivot
    
    # Save each offense type to a separate CSV
    safe_filename = offense.lower().replace('/', '_').replace(' ', '_').replace('w/', 'with_')
    filename = f"crime_{safe_filename}_by_ward_year.csv"
    filepath = os.path.join(data_folder, '../processed/', filename)
    offense_pivot.to_csv(filepath)
    print(f"Saved {offense} data to: {filename}")

# Save the comprehensive dataset
comprehensive_file = os.path.join(data_folder, '../processed/crime_by_ward_year_offense_comprehensive.csv')
crime_by_ward_year_offense.to_csv(comprehensive_file, index=False)

print(f"\nFiles created:")
print(f"- Comprehensive data: crime_by_ward_year_offense_comprehensive.csv")
print(f"- Individual offense files: {len(offense_types)} CSV files")
print(f"\nTotal offense types: {len(offense_types)}")
print(f"Available offense types: {sorted(offense_types)}")

# Display sample of comprehensive data
print(f"\nComprehensive DataFrame shape: {crime_by_ward_year_offense.shape}")
crime_by_ward_year_offense.head(15)

Creating DataFrames for 9 offense types...
Saved ASSAULT W/DANGEROUS WEAPON data to: crime_assault_w_dangerous_weapon_by_ward_year.csv
Saved THEFT/OTHER data to: crime_theft_other_by_ward_year.csv
Saved THEFT F/AUTO data to: crime_theft_f_auto_by_ward_year.csv
Saved ROBBERY data to: crime_robbery_by_ward_year.csv
Saved MOTOR VEHICLE THEFT data to: crime_motor_vehicle_theft_by_ward_year.csv
Saved SEX ABUSE data to: crime_sex_abuse_by_ward_year.csv
Saved HOMICIDE data to: crime_homicide_by_ward_year.csv
Saved BURGLARY data to: crime_burglary_by_ward_year.csv
Saved ARSON data to: crime_arson_by_ward_year.csv

Files created:
- Comprehensive data: crime_by_ward_year_offense_comprehensive.csv
- Individual offense files: 9 CSV files

Total offense types: 9
Available offense types: ['ARSON', 'ASSAULT W/DANGEROUS WEAPON', 'BURGLARY', 'HOMICIDE', 'MOTOR VEHICLE THEFT', 'ROBBERY', 'SEX ABUSE', 'THEFT F/AUTO', 'THEFT/OTHER']

Comprehensive DataFrame shape: (477, 4)


,ward,year,offense,count
0,1.0,2019,ARSON,1
1,1.0,2019,ASSAULT W/DANGEROUS WEAPON,163
2,1.0,2019,BURGLARY,135
3,1.0,2019,HOMICIDE,16
4,1.0,2019,MOTOR VEHICLE THEFT,248
5,1.0,2019,ROBBERY,407
6,1.0,2019,SEX ABUSE,17
7,1.0,2019,THEFT F/AUTO,1690
8,1.0,2019,THEFT/OTHER,2148
9,1.0,2020,ARSON,1


In [ ]:
# Create detailed homicides analysis by ward and year
homicides = crime_incidents[crime_incidents['offense'] == 'HOMICIDE']

# Create pivot table: wards as rows, years as columns
homicides_by_ward_year = homicides.groupby(['ward', 'year']).size().unstack(fill_value=0)

# Convert ward numbers to strings for better display
homicides_by_ward_year.index = homicides_by_ward_year.index.astype(str)

# Add a total column (sum across all years for each ward)
homicides_by_ward_year['Total_All_Years'] = homicides_by_ward_year.sum(axis=1)

# Add a total row (sum across all wards for each year)
totals_row = homicides_by_ward_year.sum(axis=0)
totals_row.name = 'Total_All_Wards'

# Combine the data with totals
homicides_with_totals = pd.concat([homicides_by_ward_year, totals_row.to_frame().T])

# Save to CSV
homicides_file = os.path.join(data_folder, '../processed/homicides_by_ward_year_with_totals.csv')
homicides_with_totals.to_csv(homicides_file)

print(f"Homicides data saved to: homicides_by_ward_year_with_totals.csv")
print(f"Data includes:")
print(f"- Rows: {len(homicides_by_ward_year)} wards + 1 total row")
print(f"- Columns: Years 2019-2025 + Total column")
print(f"- Total homicides across all years and wards: {homicides_with_totals.loc['Total_All_Wards', 'Total_All_Years']}")

# Display the data
print(f"\nHomicides by Ward and Year:")
homicides_with_totals

Homicides data saved to: homicides_by_ward_year_with_totals.csv
Data includes:
- Rows: 8 wards + 1 total row
- Columns: Years 2019-2025 + Total column
- Total homicides across all years and wards: 1351

Homicides by Ward and Year:


year,2019,2020,2021,2022,2023,2024,2025,Total_All_Years
1.0,16,14,13,14,28,15,7,107
2.0,0,9,7,11,11,11,4,53
3.0,3,3,2,2,2,2,2,16
4.0,6,9,20,9,16,15,3,78
5.0,18,25,25,33,36,23,12,172
6.0,12,19,11,9,26,11,12,100
7.0,46,57,62,47,56,45,23,336
8.0,65,62,82,78,99,65,38,489
Total_All_Wards,166,198,222,203,274,187,101,1351
